In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager    
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from tqdm import tqdm_notebook
from urllib.request import urlopen
import time
import requests
import os
from glob import glob
import shutil
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

전기간 관람객 수 개별 영화 관람객 수

개봉 10일차까지 보이는 공통적인 패턴.

총관람객수 추이와 개별 영화 추이의 관계 3.천만 영화 - 비천만영화

개봉요일에 따른 차이

천만영화 상영 중일 떄의 전체 관람객 수 - 아닐 떄의 전체 관람객 수

연휴와의 관계

연휴를 많이 포함하는 영화의 관람객 수
날씨와 총관람객 수의 관계

연휴, 주중/주말, 요일, 기온/강수량, 티켓 가격

In [3]:
# 전처리 

In [7]:
# 일자별 관람 통계
daily_tot_movie_aud=pd.read_csv("../movie_DB/macro_envirment/audience1_preprocess1.csv", 
                                usecols=lambda column: column != "Unnamed: 0" )
daily_aud=daily_tot_movie_aud[["날짜","전체상영편수","전체관객수","요일"]]
daily_aud.loc[:, "날짜"] = pd.to_datetime(daily_aud["날짜"], format="%Y-%m-%d %H:%M:%S")
daily_aud.columns=["일자","전체상영편수","전체관객수","요일"]
daily_aud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7036 entries, 0 to 7035
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      7036 non-null   datetime64[ns]
 1   전체상영편수  7036 non-null   int64         
 2   전체관객수   7036 non-null   int64         
 3   요일      7036 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 220.0+ KB


C:\Users\acorn\AppData\Local\Temp\ipykernel_2524\3973171098.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_aud.loc[:, "날짜"] = pd.to_datetime(daily_aud["날짜"], format="%Y-%m-%d %H:%M:%S")


In [20]:
# 개별 영화 통계를 Dict로
csvs=glob("../movie_DB/individual_trend/*.csv")
movies = {}
m_movies={}

for file_path in csvs:
    # Get the file name without the extension
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    
    movies[file_name] = pd.read_csv(file_path,usecols=lambda column: column != "Unnamed: 0")

for name in movie_titles:
    movies[f"{name}"]=movies[f"{name}"][["날짜",
                                         "스크린수","스크린점유율",
                                         "관객수","관객수증감(전일대비)"]]
    movies[name] = movies[name].rename(columns={"날짜": "일자"})
    movies[f"{name}"]["일자"] = pd.to_datetime(movies[f"{name}"]["일자"], format="%Y-%m-%d")
    m_movies[f"{name}"]=pd.merge(movies[f"{name}"],daily_aud)

In [10]:
movie_titles = list(movies.keys())
print(movie_titles)

['1987', '7번방의 선물', '검사외전', '겨울왕국 2', '겨울왕국', '공조', '과속스캔들', '관상', '광해, 왕이 된 남자', '괴물', '국가대표', '국제시장', '극한직업', '기생충', '도둑들', '디워', '명량', '미션임파서블-고스트프로토콜', '밀정', '백두산', '범죄도시2', '베를린', '베테랑', '변호인', '보헤미안 랩소디', '부산행', '설국열차', '수상한 그녀', '스파이더맨- 노 웨이 홈', '스파이더맨- 파 프롬 홈', '스파이더맨- 홈 커밍', '신과함께-인과 연', '신과함께-죄와 벌', '실미도', '써니', '아바타- 물의 길', '아바타', '아이언맨 3', '알라딘', '암살', '어벤져스- 에이지 오브 울트론', '어벤져스- 엔드게임', '어벤져스- 인피니티 워', '엑시트', '왕의 남자', '인터스텔라', '최종병기 활', '캡틴 아메리카- 시빌 워', '탑건- 매버릭', '태극기 휘날리며', '택시운전사', '트랜스포머 3', '트랜스포머- 패자의 역습', '트랜스포머', '한산- 용의 출현', '해운대', '해적- 바다로 간 산적', '히말라야']


In [21]:
m_movies["겨울왕국"].head()

,일자,스크린수,스크린점유율,관객수,관객수증감(전일대비),전체상영편수,전체관객수,요일
0,2014-01-16,676,0.149,160592,160295,78,441617,Thursday
1,2014-01-17,785,0.169,170315,9723,84,480260,Friday
2,2014-01-18,962,0.196,430189,259874,83,1063480,Saturday
3,2014-01-19,1010,0.207,435610,5421,76,1016808,Sunday
4,2014-01-20,801,0.183,157528,-278082,76,360895,Monday


In [ ]:
#